In [53]:
import plotly.express as px
import plotly.graph_objs as go
import mysql.connector
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

True

In [19]:
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database='dublindb'
)
cursor = conn.cursor()

## Histogram of contacts to searches ratio per user

In [55]:
query_booked = '''
    SELECT s.id_user,
        SUM(s.n_searches) AS total_searches,
        COUNT(DISTINCT c.id_guest, c.id_host) AS num_contacts
    FROM searches s
    LEFT JOIN contacts c ON s.id_user = c.id_guest
    WHERE c.ts_booking_at IS NOT NULL
    GROUP BY s.id_user
'''
cursor.execute(query_booked)
res_booked = cursor.fetchall()

In [56]:
query_nobooking = '''
    SELECT s.id_user,
        SUM(s.n_searches) AS total_searches,
        COUNT(DISTINCT c.id_guest, c.id_host) AS num_contacts
    FROM searches s
    LEFT JOIN contacts c ON s.id_user = c.id_guest
    WHERE c.ts_booking_at IS NULL
    GROUP BY s.id_user
'''
cursor.execute(query_nobooking)
res_nobooking = cursor.fetchall()

In [58]:
ratios_booked = [result[2] / result[1] for result in res_booked]
ratios_nobooking = [result[2] / result[1] for result in res_nobooking]

fig = go.Figure()

fig.add_trace(go.Histogram(x=ratios_booked, name='Contacts with bookings'))
fig.add_trace(go.Histogram(x=ratios_nobooking, name='Contacts without bookings'))

fig.update_layout(
    title='Histogram of contacts to searches ratio per user',
    xaxis_title='Ratio',
    yaxis_title='Count',
    barmode='overlay',
)

fig.show()

## Contact details

In [70]:
query_booked = '''
    SELECT c.n_guests, COUNT(*) AS num_contacts_booked
    FROM contacts c
    WHERE c.ts_booking_at IS NOT NULL
    GROUP BY c.n_guests
'''

query_not_booked = '''
    SELECT c.n_guests, COUNT(*) AS num_contacts_not_booked
    FROM contacts c
    WHERE c.ts_booking_at IS NULL
    GROUP BY c.n_guests
'''

cursor.execute(query_booked)
res_booked = cursor.fetchall()

cursor.execute(query_not_booked)
res_not_booked = cursor.fetchall()

guests_booked = {result[0]: result[1] for result in res_booked}
guests_not_booked = {result[0]: result[1] for result in res_not_booked}

fig = go.Figure()

fig.add_trace(go.Bar(x=list(guests_booked.keys()), y=list(guests_booked.values()), name='Booked Contacts', marker_color='blue'))
fig.add_trace(go.Bar(x=list(guests_not_booked.keys()), y=list(guests_not_booked.values()), name='Not Booked Contacts', marker_color='red'))

fig.update_layout(
    title='Count of Contacts Based on Number of Guests in Inquiry',
    xaxis_title='Number of Guests',
    yaxis_title='Count',
    barmode='group',
)

fig.show()

In [71]:
query_messages_booked = '''
    SELECT c.n_messages, COUNT(*) AS num_contacts_booked
    FROM contacts c
    WHERE c.ts_booking_at IS NOT NULL
    GROUP BY c.n_messages
'''

query_messages_not_booked = '''
    SELECT c.n_messages, COUNT(*) AS num_contacts_not_booked
    FROM contacts c
    WHERE c.ts_booking_at IS NULL
    GROUP BY c.n_messages
'''

cursor.execute(query_messages_booked)
res_messages_booked = cursor.fetchall()

cursor.execute(query_messages_not_booked)
res_messages_not_booked = cursor.fetchall()

messages_booked = {result[0]: result[1] for result in res_messages_booked}
messages_not_booked = {result[0]: result[1] for result in res_messages_not_booked}

fig = go.Figure()

fig.add_trace(go.Bar(x=list(messages_booked.keys()), y=list(messages_booked.values()), name='Booked Contacts', marker_color='blue'))
fig.add_trace(go.Bar(x=list(messages_not_booked.keys()), y=list(messages_not_booked.values()), name='Not Booked Contacts', marker_color='red'))

fig.update_layout(
    title='Count of Contacts Based on Number of Messages in Inquiry',
    xaxis_title='Number of Messages',
    yaxis_title='Count',
    barmode='group',
)

fig.show()

In [ ]:
cursor.close()
conn.close()